# Lab H Exploration: AR Markers

このノートブックでは、ArUcoライブラリを使ってカラー画像からARマーカーを検出する方法を学びます。

このノートブック全体を通して **<font style="color:red">太い赤字で書かれた文章</font>** は、実行する前にその下のコードブロックを編集して正しいコードを書く必要があります。


## 目次
1. [はじめに](#GettingStarted)
1. [ARマーカーの検出](#DetectingARMarkers)
1. [ARマーカーの整理](#OrganizingData)
1. [方向の検出](#DetectingOrientation)
1. [色の検出](#DetectingColor)

<a id="GettingStarted"></a>
## 1. はじめに

**<font style="color:red">もしシミュレータを利用して開発を進める場合は、 `isSimulation` を `True` に設定します </font>**。 実際のマシンを利用する場合は、 `isSimulation` を `False` のままにしてください。

In [ ]:
# TODO: 必要に応じてisSimulationを更新する
isSimulation = True

次に、Pythonライブラリ(`cv`, `numpy`, など)や、Racecarライブラリ(`racecar_core`)など、このノートブックの実行に必要なライブラリをインポートします。

In [ ]:
# Pythonライブラリのインポート
import math
import copy
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import statistics
from nptyping import NDArray
from typing import Any, Tuple, List, Optional
from enum import Enum

# Racecarライブラリのインポート
import sys
sys.path.append("../../library")
import racecar_core
import racecar_utils as rc_utils

以下の関数を使えば、Jupyter Notebookに画像を表示することができます。

In [4]:
def show_image(image, size = 8) -> None:
    """
    Displays a color image in the Jupyter Notebook.
    """
    plt.figure(figsize=(size, size), dpi=100)
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))

最後に、Racecarオブジェクトを作成します。このステップで失敗した場合は`isSimulation` が正しい値であることを確認してください。

In [ ]:
# Racecar オブジェクトの作成
rc = racecar_core.create_racecar(isSimulation)

<a id="DetectingARMarkers"></a>
## 2. ARマーカーの検出
QRコードと同様に、ARマーカーはコンピュータに認識されやすいように設計された特別な模様のパターンです。このコースでは、OpenCVのArUcoライブラリで検出できるArUcoマーカーを扱います。各パターンには固有のID番号が設定されており、ArUcoライブラリはマーカーがどの方向を向いていいるかを検出することができます。

まず始めに、カラー画像を取得してみましょう。

In [ ]:
# 直近のカラー画像を取得する
image = rc.camera.get_color_image_async()
show_image(image)

次に`cv.aruco.ArucoDetector.detectMarkers`関数を用いて、この画像にあるマーカーを検出します。この関数は、マーカーに関する多くの情報を含むオブジェクトを返します。これらの情報が何を意味するかは、次のセクションで説明します。

In [ ]:
# パラメータを設定する
aruco_detector = cv.aruco.ArucoDetector(
    cv.aruco.getPredefinedDictionary(cv.aruco.DICT_6X6_250), 
    cv.aruco.DetectorParameters(),
    )

# ARマーカーを検出する
ar_markers = aruco_detector.detectMarkers(image)
ar_markers

最後に`cv.aruco.drawDetectedMarkers` 関数を用いて、カラー画像中のARマーカーに注釈をつけます。この関数では、以下のパラメータが必要です：
1. マーカーが検出されたカラー画像
2. マーカーの角の座標(これは`detectMarkers`が返すオブジェクトの0番目の要素です)
3. マーカーのID(これは`detectMarkers`が返すオブジェクトの1番目の要素です)
4. 画像につける注釈の色

画像内で検出された各ARマーカーに対して以下の処理を行います：
* マーカーを指定した色の枠で囲う
* パターンの左上隅に四角形を表示させる。これはマーカーの向きを示します。つまり、マーカーが上向きでない場合、四角形はマーカーに対して左上隅にはありません。
* 'id=N'という注釈をつける。ここで$N$とはマーカーの検出された順番（インデックス）を示します。

In [ ]:
color = (255, 255, 0)
cv.aruco.drawDetectedMarkers(image, ar_markers[0], ar_markers[1], color)

show_image(image)

`drawDetectedMarkers`は、渡した画像の上に**直接**注釈を描画することに注意しましょう。画像に直接注釈を描画してしまうと、輪郭の検出など、画像に対して追加の計算を行う必要がある場合に問題となります。この場合、まず[`copy.deepcopy()`](https://docs.python.org/3/library/copy.html)関数を使って画像の*ディープコピー*を作成する必要があります。

**<span style="color:red">検出されたマーカーを描画する前に`image`のディープコピーを作成するように、以下のコードブロックを修正してください。</span>** このコードを実行すると、2つの画像が表示されるはずです。1つ目は注釈なし、2つ目は注釈付きの画像です。

In [ ]:
image = rc.camera.get_color_image_async()
ar_markers = aruco_detector.detectMarkers(image)

# TODO: 画像のディープコピーを作成する
image_copy = ?

color = (255, 255, 0)
cv.aruco.drawDetectedMarkers(image_copy, ar_markers[0], ar_markers[1], color)

show_image(image)
show_image(image_copy)

<a id="OrganizingData"></a>
## 3. ARマーカーの整理

### 3.1 データの解凍

ARマーカーをプログラムで使用するためには、`detectMarkers`関数が返す情報を理解する必要があります。

`detectMarkers`関数が返す`ar_markers`オブジェクトを表示してみると、深くネストした配列でゴチャゴチャしていることがわかります。このデータを使用するためには、有用な部分のみを抽出する必要があります。

In [ ]:
ar_markers

`ar_markers`の0番目の要素は画像内の各ARマーカーの角の座標を含む配列で、1番目の要素は画像内の各ARマーカーのIDを含む配列です。

In [ ]:
corners = ar_markers[0]
ids = ar_markers[1]

In [ ]:
corners

In [ ]:
ids

残念ながら、角の座標はRacecarのライブラリと互換性のあるフォーマットで表現されていません。まず第一に、各角の座標は（ピクセル行、ピクセル列）の順ではなく、（ピクセル列、ピクセル行）の順で表現されています。第二に各行と各列は`float32`（端数を許可する）浮動小数点表現で保存されています。これは、今回のアプリケーションではあまり意味がありません。なので、各行と各列の値は`int32`に変換したいです。

データは深くネストされた配列に格納されているので、必要なデータにアクセスするにはこれらの配列を「アンパック」する必要があります。例えば、`ids`配列の各要素は、それ自体がIDを含む1要素の配列です。したがって、最初のARマーカーのIDは`id[0][0]`で、次のARマーカーのIDは`ids[1][0]`で取得することができます。

**<span style="color:red">最初に検出されたARマーカー（インデクス0）の角の座標とIDを抽出するために、以下のコードブロックを修正してください。</span>** なお、角の座標は（行、列）の順で、整数値で表現してください。

In [ ]:
# 最初に検出されたARマーカーの角の座標を取得して、データ型をfloat32からint32に変換する
first_corners = corners[0][0].astype(np.int32)

# TODO: 各角の座標を(col, row)から(row, col)に入れ替える


# TODO: first_idを最初に検出されたARマーカーのIDに設定する
first_id = ?

print("first_corners: ", first_corners)
print("first_id: ", first_id)

### 3.2 ARマーカークラスの作成
ARマーカーの角の座標とIDを別々に保存するのではなく、1つのマーカーに関するすべての情報を保存する1つのオブジェクトがあると便利です。このようなオブジェクトを作成しておくことは、後でマーカーの向きと色を計算するときに役立ちます。

そのためには、新しいタイプのオブジェクトを指定する*クラス(class)*を作成します。例えば、`ARMarker`クラスを定義すると、`ARMarker`型の新しいオブジェクトを作成することができます。

クラス(class)は、以下の2つの要素で構成されています：
1. **フィールド(Fields)**：フィールドはクラスに関連するデータを格納する変数です。ここでは、IDを格納する`id`フィールドと、マーカーの角の座標を格納する`corners`フィールドを作成します。
2. **メソッド(Messods)**：メソッドはクラスのオブジェクトに対して呼び出すことができる関数の一種です。メソッドの最初のパラメータ(引数)は`self`と呼ばれ、メソッドが呼び出されたオブジェクトの事をさします。

`__init__`メソッドは _コンストラクタ(constructor)_ と呼ばれる特別なメソッドで、クラスの新しいオブジェクトを作成するために使用されます。伝統的には、ここでクラスのフィールドを定義します。ここでは`ARMarker`コンストラクタを記述して、マーカーIDと角の座標を受け取ります。

**<span style="color:red">`ARMarker`クラスのコンストラクタを作成して、`id`と`corners`フィールドに正しい値を代入しましょう。</span>**

In [ ]:
class ARMarker:
    
    def __init__(self, marker_id, marker_corners):
        # TODO: idフィールドにmarker_idパラメータを割り当てる
        self.id = ?
            
        # TODO: cornersフィールドにmarker_cornersパラメータを割り当てる
        self.corners = ?

このコンストラクタを呼び出すと、前に計算したidとcornerを格納した`ARMarker`オブジェクトが作成されます。第一パラメータに`self`が存在しますが、引数を渡す必要がないことに注意しましょう。

In [ ]:
# 最初に検出されたARマーカーんおIDと角の座標を格納するARMarkerオブジェクトを作成する
first_marker = ARMarker(first_id, first_corners)

# 最初に検出されたARマーカーのIDと角の座標を表示する
print("first_marker.id: ", first_marker.id)
print("first_marker.corners: ", first_marker.corners)

先の例では、オブジェクトのフィールドを読み取る方法（first_marker.idやfirst_marker.corners）を試しました。もちろん、オブジェクトのフィールドに直接書き込む事もできます。

In [ ]:
first_marker.id = 20
print("first_marker.id: ", first_marker.id)

### 3.3 プライベートフィールド (Private Fields)

`ARMarker`クラスの最初に検出された後にIDや角の座標をユーザーが変更できるようにする必要はありませんし、勝手に変更されると困ってしまいます。そのため、これらのフィールドの名前の前に2つアンダースコア(`__`)をつけることで、そのフィールドを*プライベート(private)*に設定することができます。プライベートに設定されたフィールドは`ARMarker`クラスのメソッドのみからアクセスでき、クラスの外部からはアクセスできなくなります。

それでは、IDと角の座標用のフィールドを、`__id`と`__corners`を書き直して、ユーザーが変更できないように修正しましょう。また、ユーザーが`__id`と`__corners`の値を変更せずにアクセスできるように、`get_id()`メソッドと`get_corners()`メソッドを追加しましょう。このようにフィールドの値を返すメソッドは _accessors_ または _getters_ と呼ばれます。

In [ ]:
class ARMarker:
    
    def __init__(self, marker_id, marker_corners):
        # TODO: __id と __corners フィールドを定義する

            
    def get_id(self):
        # TODO: ARマーカーのIDを返す

    
    def get_corners(self):
        # TODO: ARマーカーの角の座標を返す


先ほどと同じようにARMarkerオブジェクトが作成します。ただし、IDと角の座標には、`get_id()`メソッドと`get_corners()`メソッドを使ってアクセスします。ここでも、引数`self`を明示的に渡さないことに注意してください。

In [ ]:
first_marker = ARMarker(first_id, first_corners)

print("first_marker.get_id(): ", first_marker.get_id())
print("first_marker.get_corners(): ", first_marker.get_corners())

`__id`はプライベートフィールドなので、`first_marker.__id`でアクセスすることはできません。以下のコードブロックを実行して、ユーザーが`__id`の値が変更できないことを確認してください。これにより、ユーザーが誤ってマーカーのIDを上書きしてしまうことを防ぐ事ができます。

In [ ]:
first_marker.__id = 20
print("first_marker.get_id(): ", first_marker.get_id())

### 3.4 `__str__`の追加

今のところ、`ARMarker`オブジェクトをprintで表示しようとしても、オブジェクトがメモリのどこに保存されているかを教えてくれるだけで、特段役に立ちません。

In [ ]:
print(first_marker)

これを解決するために、`ARMarker`クラスに`__str__`を実装してみましょう。`__str__`はオブジェクトを文字列に変換するための特別なメソッドです。このメソッドを実装すると、オブジェクトを`print()`で表示したり、`str()`で文字列に変換したりする際に自動で呼び出されます。

**<span style="color:red">マーカーのIDと角の座標を表示するための`__str__`メソッドを完成させましょう。</span>**

In [ ]:
class ARMarker:
    
    def __init__(self, marker_id, marker_corners):
        # TODO: これまでに作成したARMarkerクラスからコピーする

            
    def get_id(self):
        # TODO: これまでに作成したARMarkerクラスからコピーする

    
    def get_corners(self):
        # TODO: これまでに作成したARMarkerクラスからコピーする

    
    def __str__(self):
        # TODO: マーカーのIDと角の座標を文字列で返す


`__str__`メソッドを実装したことで、`print()`関数で`ARMarker`オブジェクトを表示すると、より親切なメッセージが表示されます。

In [ ]:
first_marker = ARMarker(first_id, first_corners)
print(first_marker)

### 3.5 一つにまとめる

**3.1**節では、画像の中のマーカーの角の座標情報とID情報を抽出する方法を学びました。**3.2 - 3.4**節では、1つのARマーカーについて、角の座標とIDの情報を整理するために`ARMarker`クラスを作成しました。このセクションでは、これらをまとめて、画像内のすべてのARマーカーを抽出して`ARMarker`オブジェクトのリストとして返す関数を作成します。

**<span style="color:red">画像から検出されたARマーカーの情報をまとめた`ARMarker`オブジェクトのリストを返す、`get_ar_markers()`関数を完成させましょう。</span>** ArUcoライブラリは角の座標を(col, row)のフォーマットで、`float32`の値として表現しますが、私たちは(row, col)のフォーマットで`int32`の値として表現したいことを忘れないでください。わからなくなったら**3.1**節のコードをもう一度見直してみてください。

In [ ]:
def get_ar_markers(image):
    # ArUcoから生のARマーカーデータを収集する
    aruco_data = aruco_detector.detectMarkers(image)
    
    # aruco_dataで見つかったARマーカーを表すARMarkerオブジェクトのリスト
    markers = []
        
    for i in range(len(aruco_data[0])):
        # TODO: aruco_dataの各マーカーについて、角の座標とIDを抽出して、
        # 角の座標を(row, col)のフォーマットに変更し、このデータでARMarkerオブジェクトを作成する(3.1節を参照)
        
        # TODO: 新しいマーカーをmarkersリストに追加する

        
    return markers

これで、`get_ar_markers()`を使って画像内のすべてのARマーカーを特定できるようになりました。

In [ ]:
image = rc.camera.get_color_image_async()
markers = get_ar_markers(image)

for marker in markers:
    print(marker)
    print("\n----\n")

<a id="DetectingOrientation"></a>
## 4. 方向の検出

ARマーカーのパターンは左右対称でないため、どのように回転させても、ArUcoは常にパターンの「一番上」を識別することができます。ArUcoが返す角の座標データは、常にARマーカーパターンの左上の角から始まり、時計回りに続きます。

この情報を使って、画像内のARマーカーの向きを判断する事ができます。例えば、マーカーが右を向いているときは右折し、左を向いているときは左折する、といった事も実現できるようになります。

4つの方向（上、左、下、右）を表現するために、`Enum(列挙型)`を作成します。Enumは特別なタイプのクラスで、各メンバーはユニークな低数値を持っています。これらのメンバは通常、Enumクラスで表されるカテゴリの様々なオプションを表すために使用されます。

次のコードブロックでは、4つの方向（上、左、下、右）を表すメンバを持つ`Orientation`列挙型(Enum)を定義しています。各メンバは0から3までの一意な低数値を持っています。

In [ ]:
class Orientation(Enum):
    UP = 0
    LEFT = 1
    DOWN = 2
    RIGHT = 3

列挙型(Enum)にはいくつかの役立つ性質があります：
* Enum値は変数に格納できます。例えば、マーカーの向きを格納する変数を定義することができます。
* Enumの値は`==`で直接比較することができ、Enumの値が同じであれば`true`を返し、そうでなければ`false`を返します。例えば、`Orientation.UP == Orientation.UP`は`true`を返し、`Orientation.Up == Orientation.DOWN`は`false`を返します。
* Enumの値には`.value`でアクセスする事ができます。例えば`Orientation.UP.value`は0で、`Orientation.LEFT.value`は3です。

In [ ]:
# 列挙型の値は変数に格納する事ができる
marker_orientation = Orientation.UP

# 列挙型の値を比較できる
if (marker_orientation == Orientation.DOWN):
    print("marker_orientation is DOWN")
if (marker_orientation == Orientation.UP):
    print("marker_orientation is UP")
    
# 列挙型の値にはvalueでアクセスできる
print("marker_orientation.value is", marker_orientation.value)

次に、マーカーの向きを自動的に検出して保存するために、`ARMarker`クラスを拡張します。`marker_corners`は常にパターンの左上の角の座標を最初に持ち、そこから時計回りに残りの角の座標を並べています。例えば、マーカーが右向き（時計回りに90度回転）の状態であれば、画像におけるのマーカの左上の角の座標は`marker_corners[3]`となります。コーナーの行と列を比較することで、マーカーの向きを判断する事ができます。（最初に図を描いて、マーカーの実際の左上と回転、座標の関係をまとめることを推奨します）

**<span style="color:red">次のコードブロックでは、`ARMarker`クラスに以下の変更を加えてください。</span>**
* はじめに**3.4**節の実装をコピーしてください。
* `__init__`メソッドに、`__orientation`フィールドの定義を新たに追加し、画像内のマーカーの向きを格納します。このフィールドには`Orientation`列挙型の値を格納します。
* 新しい`get_orientation`メソッドを実装して、マーカーの向きを返すようにします。
* マーカーの向きも表示されるように`__str__`メソッドを修正してください。

In [ ]:
class ARMarker:
    
    def __init__(self, marker_id, marker_corners):
        # TODO: 以前のARMarkerクラスから__idと__cornersの実装をコピーする

              
        # TODO: ARマーカの方向の情報を示す__orientationフィールドを設定する
        # marker_cornersで渡された座標情報からARMarkerの向きを検出します
        
    
 
            
    def get_id(self):
        # TODO: 以前のARMarkerクラスから実装をコピーする

    
    def get_corners(self):
        # TODO: 以前のARMarkerクラスから実装をコピーする。

    
    def get_orientation(self):
        # TODO: ARマーカーの方向を返す

    
    def __str__(self):
        # TODO: __str__を修正し、ID、コーナー、方向を返すようにする


ARマーカーの方向検出アルゴリズムは`ARMarker`コンストラクタにあるので、ARMarkerを作成するたびに自動的に実行されます。したがって、`get_ar_markers`関数を修正する必要はありません。クラスを使用するすべてのコードを変更することなく、クラスを変更する事ができます。このように、変更しなくてはいけないコードの範囲を狭くできるのは、クラスを使用する大きなメリットです。

次のコードブロックを実行すると、すべてのARマーカーの向きもIDや角の座標と合わせて表示されるはずです。

In [ ]:
image = rc.camera.get_color_image_async()
markers = get_ar_markers(image)

for marker in markers:
    print(marker)
    print("\n----\n")

<a id="DetectingColor"></a>
## 5. 色の検出
ラインフォローでは、あらかじめわかっている色の優先順位に基づいて色のついた線をたどりました。より難しいタスクとしては、環境にある指標に基づいて色の優先順位を決定することが考えられます。

色の情報を伝える一つの方法は、色のついた背景にARマーカーを表示する事です。例えば、赤色の背景にARマーカーを表示すれば、赤い線に沿って進めば良いことを示す事ができます。また、複数のマーカーを使って、色の優先順位を伝えることもできます。マーカーのID 0の色が最も優先順位の高い色、マーカーID 1の色がその次の色、という具合です。

この情報を利用するには、ARマーカーを囲む背景色を検出するアルゴリズムを実装する必要があります。ラインフォローで学んだ事を応用すれば、以下のステップでこの機能を実現することができます：
1. 画像をARマーカーの周囲で切り抜く
2. 切り取られた領域内で、色の輪郭を検出する
3. 最も大きな輪郭が得られた色は、マーカーを囲む背景色である可能性が高い

### 5.1 マーカーのトリミング

マーカーの角の座標と向きから、画像に対する左上と右下の角の座標を見つけることができます。しｓかし、この領域でトリミングすると、マーカーそのものが見えるだけで、マーカーを囲む背景色は見えません。なので、マーカーの左上と右下の角の座標情報からマーカーの幅と高さを取得して、それを使ってARマーカーの2倍の大きさの四角形を考えます。この四角形の左上の角の座標(top_left)と右下の角の座標(bottom_right)を計算します。そして、`rc_utils.crop`を使うことで、この四角形に囲まれた画像を切り抜くことができます。

**<span style="color:red">以下のコードブロックを、`image`からARマーカーを中心とした、ARマーカーの高さと幅が2倍の領域を切り抜けるように追記・修正しましょう。</span>**

**注 1**: マーカーの高さを計算するよりも、半分の高さを計算するほうが処理を簡単に記述できます。なぜなら、求める2倍の大きさの四角形は、ARマーカーの左上の座標を高さの半分の値だけ上に、左下の角を高さの半分の値だけ下にずらす事で求める事ができるためです。幅についても同じように考える事ができます。（実装する前に図を描いて考えてみましょう）

**注 2**: マーカーの角の座標として返ってくる値は*パターン*の左上の角の座標であり、 _image_ に表示されている左上の角の座標とは限らないことに注意してください。幸運なことに、4節ですでにマーカーの向きを計算する機能を実装しています。それぞれの`Orientation`の値には(`UP`には0が、`LEFT`には1のように)整数の値が関連づけられていることを思い出してください。この整数の値は、マーカーの角の座標のリストにおける、画像に対する左上のコーナーのインデックスも表す値です。つまり、マーカーが左を向いている場合、`markers.get_orientation()[1]`の値は左上の角の座標を示します。

In [ ]:
# 画像内の最初のマーカーを検出
image = rc.camera.get_color_image_async()
markers = get_ar_markers(image)
marker = markers[0]

# 画像内のマーカーの位置と寸法を計算する(注2を参照)
marker_top, marker_left = marker.get_corners()[marker.get_orientation().value]
marker_bottom, marker_right = marker.get_corners()[(marker.get_orientation().value + 2) % 4]

# TODO: ARマーカーの高さの半分の値と幅の半分の値を計算する(注1を参照)
half_marker_height = ?
half_marker_width = ?

# TODO: ARマーカーを中心に、ARマーカーの2倍の大きさの四角形を切り抜く


# 切り抜いた画像を表示する
show_image(cropped_image, 4)

### 5.2 輪郭を見つける
次に、切り取った画像から色の輪郭を探します（ラインフォローの時の手順を参考にしてください）。まず、赤の輪郭を探します。**<span style="color:red">必要であれば、切り取られた赤い背景を識別できるまで、以下のHSV範囲を変更してください。</span>**

In [ ]:
# TODO: 必要に応じて、マーカーの輪郭を検出するHSV範囲を調整する
hsv_lower = (170, 50, 50)
hsv_upper = (10, 255, 255)

# 最も大きな輪郭を検出する
contours = rc_utils.find_contours(cropped_image, hsv_lower, hsv_upper)
largest_contour = rc_utils.get_largest_contour(contours)

# 切り取った画像のコピーに輪郭を描き、それを表示する
marked_image = copy.deepcopy(cropped_image)
rc_utils.draw_contour(marked_image, largest_contour)
show_image(marked_image, 4)

しかし、検出したいのは赤い輪郭だけではありません。ARマーカーが青、緑、赤のいずれかであることがわかっているとします。それぞれを検出できるようにしましょう。各色のHSV範囲に対して、これまでの戦略を適用することで、最も大きな輪郭を持つ色を選択することができます。輪郭が見つからなければ、ARマーカーは青、緑、赤のいずれの色でもないと結論付けます。

**<span style="color:red">`detected_color`に、輪郭の面積が最も大きい色の名前が格納されるように、以下のコードブロックを追記・修正してください。</span>**

In [ ]:
# TODO: 必要に応じて色やhsv_rangesを調整する
potential_colors = [
    ((90, 50, 50), (120, 255, 255), "blue"),
    ((40, 50, 50), (80, 255, 255), "green"),
    ((170, 50, 50), (10, 255, 255), "red")
]

greatest_area = 0
detected_color = "none"

for (hsv_lower, hsv_upper, color_name) in potential_colors:
    # 現在の色の最大の輪郭を見つける
    contours = rc_utils.find_contours(cropped_image, hsv_lower, hsv_upper)
    largest_contour = rc_utils.get_largest_contour(contours)
    
    if largest_contour is not None:
        # 輪郭が存在する場合は輪郭の面積を求める
        contour_area = rc_utils.get_contour_area(largest_contour)
        
        if contour_area > greatest_area:
            # TODO: このような条件の場合どのような処理をすべきでしょうか？

            
print("greatest_area: ", greatest_area)
print("detected_color: ", detected_color)

### 5.3 ARMarkerクラスに色を追加する

最後に、この新しい機能を`ARMarker`クラスに追加して、色がマーカーの他の情報と一緒に保存されるようにします。

**<span style="color:red">次のコードブロックでは、`ARMarker`クラスに以下の変更を加えてください。</span>**
* **4**節から以前の実装をコピーしてください。
* `__init__`メソッドに、次の2つを追加してください。`__color`は現在検出されている色の名前を格納する文字列です。このフィールドは「未検出」として初期化する必要があります。`__color_area`は現在検出されている色の輪郭領域を格納する数値です。（初期化方法は各自で決定してください）
* 新しい`detec_colors`メソッドはマーカーを囲む潜在的な色のリストの検出を試み、その結果に応じて`__color`と`__color_area`フィールドを更新します。このメソッドはセクション**5.1**と**5.2**で説明したものを使用して実装してください。
* 新しい`get_color`メソッドを実装し、現在検出しているマーカーの色を返す。
* マーカーの色も表示するように`__str__`を更新してください。

In [ ]:
class ARMarker:
    
    def __init__(self, marker_id, marker_corners):
        # TODO: これまでに実装したARMarkerクラスの内容をコピーする

                
        # 検出された色と、その色の輪郭の領域を格納するフィールドを定義する

        
    def detect_colors(self, image, potential_colors):
        # TODO: これまでに書いたコードをコピーして、ARマーカーと周辺の画像をトリミングする
        
        
        # TODO: potential_colorsで色を検索するために、以前に書いたコードをコピーする

            
    def get_id(self):
        # TODO: これまでに実装したARMarkerクラスの内容をコピーする

    
    def get_corners(self):
        # TODO: これまでに実装したARMarkerクラスの内容をコピーする

    
    def get_orientation(self):
        # TODO: これまでに実装したARMarkerクラスの内容をコピーする

    
    def get_color(self):
        # TODO: 検出した色を返す

    
    def __str__(self):
        # TODO: __str__を修正して、ID、角の座標、向き、色を返すようにする


これで、新しい`detect_colors`メソッドを使って、カラー画像の各ARマーカーの背景色を検出できるようになった。

In [ ]:
image = rc.camera.get_color_image_async()
markers = get_ar_markers(image)

for marker in markers:
    marker.detect_colors(image, potential_colors)

for marker in markers:
    print(marker)
    print("\n----\n")

`lab_h.py`ではこれらのスキルを使って、ARマーカーによる指示に基づいて、コースをナビゲートします。幸運を祈ります！もし、何か質問があれば遠慮なく声をかけてください！